In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os

# List files in the current directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)


In [3]:
from tensorflow import keras

# Load the model
model = keras.models.load_model('/kaggle/input/model1/model.h5')


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import numpy as np
import math
from PIL import Image
import time

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

Trasfer file to working directory

In [8]:
import shutil

file_path = "/kaggle/input/noise-cancellation-dataset/NoisySpeech_training_singleNoise1/NoisySpeech_training_singleNoise1/noisy101_SNRdb_0.0_clnsp101.wav"
new_directory = "/kaggle/working/"

shutil.move(file_path, new_directory)



OSError: [Errno 30] Read-only file system: '/kaggle/input/noise-cancellation-dataset/NoisySpeech_training_singleNoise1/NoisySpeech_training_singleNoise1/noisy101_SNRdb_0.0_clnsp101.wav'

In [9]:
import tensorflow as tf
from tensorflow import keras
model = tf.keras.models.load_model('/kaggle/input/model1/model.h5')
import numpy as np
import math
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)

phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
#     print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals
from PIL import Image
def generateSpectrogramForWave(signal):
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width):
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2)
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real)
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude
            phasePixels[height-h-1,w] = phase
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]
    
#     print(phaseMax,phaseMin)

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
#     print(magnitudeVals,phaseVals)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered
rate = 16000
filename = "/kaggle/working/noisy101_SNRdb_0.0_clnsp101.wav"

from scipy.io import wavfile
import scipy.signal as sps
sampling_rate, data = wavfile.read(filename)
number_of_samples = round(len(data) * float(rate) / sampling_rate)
data = sps.resample(data, number_of_samples)
data = np.asarray(data, dtype=np.int16)
wavfile.write(filename,rate,data)
filename1 = '/kaggle/working/'
from pydub import AudioSegment
from pydub.utils import make_chunks
import time


myaudio = AudioSegment.from_file(filename)
chunk_length_ms = 1000 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
    
for i, chunk in enumerate(chunks):
        
    chunk_name = "chunk{0}.wav".format(i)
    name = filename1+'/'+chunk_name
    
    chunk.export(name, format="wav")
    
    rate, data = wavfile.read(name)
    
    
    if len(data.shape) >= 2 and data.size > 0:
        if data.shape[-1] > 1:
            data = data.mean(axis=-1)
        else:
            data = np.reshape(data, data.shape[:-1])
    img = generateSpectrogramForWave(data)
    np.save(filename1+'/'+chunk_name[:-4]+'.npy', img) # save
model.summary()
ROW = 257
COL = 62

test_pred = []

import glob
import os
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm
count = 0
rate = 16000

for i, _ in enumerate(chunks):
    filename = '/kaggle/working/'+'chunk{0}'.format(i)+'.npy'
    img_test = np.load(filename)
    row_,col_,_ = img_test.shape
    
    if col_ < COL:
        continue
    
    print(img_test.shape)
    
    img_test = img_test/255
    img_test = img_test.reshape(-1, ROW,COL,3)
    decoded_imgs = model.predict(img_test) #predict
    decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
    decoded_imgs = decoded_imgs*255
    decoded_imgs = decoded_imgs.astype(np.int16)
    data = recoverSignalFromSpectrogram(decoded_imgs) # save predicted audio
    file = "/kaggle/working/" +"testpred_{}".format(count)+'.wav' #created file
    scipy.io.wavfile.write(file, rate, data) #wrote the file with 'data'
    test_pred.append(file) #saves file in array
    count = count+1
test_pred[3]
from pydub import AudioSegment

sound = 0
path_recovered = '/kaggle/working/recovered/'
if not os.path.exists(path_recovered):
    os.makedirs(path_recovered)
    
for i in range(len(test_pred)):
    print(test_pred[i])
    sound += AudioSegment.from_wav(test_pred[i])
sound.export(path_recovered+"soundJoined1.wav", format="wav")

0.19s
0.19s
0.19s
0.19s
0.20s
0.19s
0.20s
0.19s
0.19s
0.19s
0.19s
0.19s
0.19s
0.19s
0.06s
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 257, 62, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 block_1_conv (Conv2D)          (None, 257, 62, 64)  1792        ['input_2[0][0]']                
                                                                                                  
 block_1_lrelu (LeakyReLU)      (None, 257, 62, 64)  0           ['block_1_conv[0][0]']           
                                                                                                  
 b

<_io.BufferedRandom name='/kaggle/working/recovered/soundJoined1.wav'>